# December 18, 2021

https://adventofcode.com/2021/day/18

In [1]:
import pandas as pd
import numpy as np

In [2]:
with open("../data/2021/18.txt", "r") as f:
    data = f.read()
data = data.split("\n")

In [3]:
test1 = ["[[[[4,3],4],4],[7,[[8,4],9]]]", "[1,1]"]
test2 = [ "[1,1]", "[2,2]", "[3,3]", "[4,4]"]
test3 = test2 + ["[5,5]"]
test4 = test3 + ["[6,6]"]

test5 = [
    "[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]"
    , "[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]"
    , "[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]"
    , "[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]"
    , "[7,[5,[[3,8],[1,4]]]]"
    , "[[2,[2,2]],[8,[8,1]]]"
    , "[2,9]"
    , "[1,[[[9,3],9],[[9,0],[0,7]]]]"
    , "[[[5,[7,4]],7],1]"
    , "[[[[4,2],2],6],[8,7]]"
]

In [4]:
#explosion tests
etests = [
    "[[[[[9,8],1],2],3],4]"
    , "[7,[6,[5,[4,[3,2]]]]]"
    , "[[6,[5,[4,[3,2]]]],1]"
    , "[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]"
    , "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"
]

In [5]:
# magnitude tests
mtests = [
     "[[1,2],[[3,4],5]]"
    ,"[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"
    ,"[[[[1,1],[2,2]],[3,3]],[4,4]]"
    ,"[[[[3,0],[5,3]],[4,4]],[5,5]]"
    ,"[[[[5,0],[7,4]],[5,5]],[6,6]]"
    ,"[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]"
]

In [6]:
final_test = [
    "[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]"
    , "[[[5,[2,8]],4],[5,[[9,9],0]]]"
    , "[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]"
    , "[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]"
    , "[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]"
    , "[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]"
    , "[[[[5,4],[7,7]],8],[[8,3],8]]"
    , "[[9,3],[[9,9],[6,[4,9]]]]"
    , "[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]"
    , "[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"
]

# Part 1

In [7]:
class Regular:
    def __init__( self, value, pos=None, parent=None, depth=1 ):
        self.value = value
        self.pos = pos
        self.depth = depth
        self.parent = parent
        self.__type__ = "Regular"

    def magnitude( self ):
        return self.value
    def increment_depth( self ):
        self.depth += 1
    def decrement_depth( self ):
        self.depth -= 1

    # These make syntax for Pair class cleaner
    def get_leftmost_object( self ):
        return self
    def get_rightmost_object( self ):
        return self
    def explode( self ):
        return False
    def split( self ):
        if self.value < 10:
            return False
        lv = int( self.value / 2 ) # round down
        rv = int( self.value / 2 + .5 ) # round up

        # replace this object with a Pair, whose children are REgular with left- and right- values
        new_pair = Pair( None, None, pos=self.pos, parent=self.parent, depth=self.depth )
        new_pair.left  = Regular( lv,  "left", parent = new_pair, depth = self.depth+1 )
        new_pair.right = Regular( rv, "right", parent = new_pair, depth = self.depth+1 )

        if self.pos == "right":
            self.parent.right = new_pair
        else:
            self.parent.left = new_pair
        return True
    
    def __str__(self):
        return str(self.value)
    def __repr__(self):
        return str(self.value)
    def print2( self ):
        return str(self.value)
        #return str(self.value) + "{" + str(self.depth) + ""}"


In [8]:
class Pair:
    def __init__( self, left, right, pos=None, parent=None, depth=1 ):
        # left and right can be Pairs or Regulars
        self.left = left
        self.right = right
        self.parent = parent
        if parent is None:
            self.pos = None
        else:
            assert pos == "left" or pos == "right", "pos must be left/right if parent is not None"
            self.pos = pos
        self.depth = depth
        self.__type__ = "Pair"

    def from_str( text, pos=None, parent=None, depth=1 ):
        obj = Pair( left=None, right=None, pos=pos, parent=parent, depth=depth )

        if text[0] == "[":
            # nested pair. find the two halves
            opens = 0
            pos = 0
            while True:
                pos += 1
                if text[pos] == "[":
                    opens += 1
                elif text[pos] == "]":
                    opens -= 1
                if opens == 0:
                    break
            # end pos hunt

            lstr = text[1:pos+1]
            rstr = text[pos+2:-1]

            obj.left = Pair.from_str(lstr, "left", parent=obj, depth=depth+1)
            obj.right = Pair.from_str(rstr, "right", parent=obj, depth=depth+1)

            return obj
        
        else:
            # otherwise this is a value. We construct and return it
            return Regular( int(text), pos=pos, parent=parent, depth=depth )
        
    def magnitude( self ):
        return 3*self.left.magnitude() + 2*self.right.magnitude()
    def increment_depth( self ):
        self.depth += 1
        self.left.increment_depth()
        self.right.increment_depth()
    def decrement_depth( self ):
        self.depth -= 1
        self.left.decrement_depth()
        self.right.decrement_depth()
    

    def __str__( self ):
        return "[" + str(self.left) + "," + str(self.right) + "]"
    def __repr__( self ):
        return str(self)
    def print2( self ):
        return " <" + str(self.depth) + ">[" + self.left.print2() + "," + self.right.print2() + "]"
    
    def __add__(self, other):
        combo = Pair(None, None)

        self.increment_depth()
        self.pos = "left"
        self.parent = combo
        combo.left = self

        other.increment_depth()
        other.pos = "right"
        other.parent = combo
        combo.right = other

        combo.reduce()
        return combo
    
    # Reduce
    def reduce( self ):
        #print(self)
        while self.explode() or self.split():
            # print(self)
            pass

    # Split function
    def split( self ):
        # yay! Boolean shortcircuiting! right.split only called if left.split is False
        return self.left.split() or self.right.split()
    # Explosion functions
    def get_righthand_neighbor( self ):
        # If we've reached the top, then there's no righthand neighbor
        if self.parent is None:
            return None
        # If this is the left side, get the leftmost object of the right side of the pair
        if self.pos == "left":
            return self.parent.right.get_leftmost_object()
        # Otherwise, go up another level
        if self.pos == "right":
            return self.parent.get_righthand_neighbor()
    # Iterate to the left until we hit a regular Object
    def get_leftmost_object( self ):
        return self.left.get_leftmost_object()
    
    def get_lefthand_neighbor( self ):
        # If we've reached the top, then there's no lefthand neighbor
        if self.parent is None:
            return None
        # If this is the right side, get the rightmost object of the left side of this pair
        if self.pos == "right":
            return self.parent.left.get_rightmost_object()
        if self.pos == "left":
            return self.parent.get_lefthand_neighbor()
    # iterate to the right until we hit a Regular object
    def get_rightmost_object( self ):
        return self.right.get_rightmost_object()
    
    def explode( self ):
        # Not deep enough to explode, possibly explode children
        if self.depth < 5:
            # explode functions return bool
            # short circuiting prevents right from exploding if left does.
            if self.left.explode():
                return True
            elif self.right.explode():
                return True
            else:
                # if neither pair exploded, we must return False
                return False
            
        # We've reached a 5-depth Pair. It must be explosioned! >=D
        lnbr = self.get_lefthand_neighbor()
        if lnbr is not None:
            lnbr.value += self.left.value
        rnbr = self.get_righthand_neighbor()
        if rnbr is not None:
            rnbr.value += self.right.value
        if self.pos == "left":
            self.parent.left = Regular(0, "left", self.parent, self.depth)
        else:
            self.parent.right = Regular(0, "right", self.parent, self.depth)
        return True               

In [9]:
x = Pair.from_str(etests[-1])
print(x)
x.print2()

[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]


' <1>[ <2>[3, <3>[2, <4>[8,0]]], <2>[9, <3>[5, <4>[4, <5>[3,2]]]]]'

In [10]:
x.explode()
x.print2()

' <1>[ <2>[3, <3>[2, <4>[8,0]]], <2>[9, <3>[5, <4>[7,0]]]]'

In [11]:
# for complte reduction, last two output are the same
for text in etests:
    x = Pair.from_str(text)
    x.reduce()
    print(x)

[[[[0,9],2],3],4]
[7,[6,[5,[7,0]]]]
[[6,[5,[7,0]]],3]
[[3,[2,[8,0]]],[9,[5,[7,0]]]]
[[3,[2,[8,0]]],[9,[5,[7,0]]]]


In [12]:
# addition tests!
left = Pair.from_str(test1[0])
right = Pair.from_str(test1[1])
print(left)
print(right)

[[[[4,3],4],4],[7,[[8,4],9]]]
[1,1]


In [13]:
tot = left+right
tot.print2()

' <1>[ <2>[ <3>[ <4>[0,7],4], <3>[ <4>[7,8], <4>[6,0]]], <2>[8,1]]'

In [14]:
def add_numbers( nums ):
    tot = Pair.from_str(nums[0])
    for i in range(1, len(nums)):
        right = Pair.from_str(nums[i])
        tot = tot + right
    return tot

In [15]:
add_numbers(test1)

[[[[0,7],4],[[7,8],[6,0]]],[8,1]]

In [16]:
add_numbers(test2)

[[[[1,1],[2,2]],[3,3]],[4,4]]

In [17]:
add_numbers(test3)

[[[[3,0],[5,3]],[4,4]],[5,5]]

In [18]:
add_numbers(test4)

[[[[5,0],[7,4]],[5,5]],[6,6]]

In [19]:
add_numbers(test5)

[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]

In [20]:
for text in mtests:
    x = Pair.from_str(text)
    print(x.magnitude())

143
1384
445
791
1137
3488


In [21]:
tot = add_numbers( final_test )
tot.magnitude()

4140

In [22]:
tot = add_numbers( data )
tot.magnitude()

4457

In [23]:
def biggest_pair_sum( nums ):
    imax = None
    jmax = None
    mmax = -1

    for i in range(len(nums)):
        for j in range(len(nums)):
            if i == j:
                continue

            left = Pair.from_str(nums[i])
            right = Pair.from_str(nums[j])
            tot = left + right
            if tot.magnitude() > mmax:
                mmax = tot.magnitude()
                imax = i
                jmax = j
    return mmax, imax, jmax

In [24]:
biggest_pair_sum( final_test )

(3993, 8, 0)

In [25]:
biggest_pair_sum( data )

(4784, 52, 85)